# 1.c LLM with tools

In this notebook you will see:
- How to create a tool
- How to enable an LLM to use it

# Setup

In [1]:
import sys
from typing import Any
import json

from loguru import logger

from conversational_toolkit.llms.base import LLMMessage, Roles
from conversational_toolkit.tools.base import Tool
from conversational_toolkit.llms.openai import OpenAILLM

In [2]:
# Remove logging
logger.remove()
logger.add(sys.stderr, level="ERROR", filter=lambda record: record["level"].no < 40)

1

# Define a tool

The first step is to create a tool that the LLM will be able to use.

For each tool, one has to implement it's core logic, but also explain what it does, plus specify it's input architecture. So that the LLM understands what is the tool for, and how to use it.

Hint: It's important to raise errors as it might help the LLM to call themselves.


```python
class FunctionDescription(TypedDict):
    name: str
    description: str
    parameters: dict[str, Any]


class ToolDescription(TypedDict):
    type: Literal["function"]
    function: FunctionDescription


class Tool(ABC):
    name: str
    description: str
    parameters: dict[str, Any]

    @abstractmethod
    async def call(self, args: dict[str, Any]) -> dict[str, Any]:
        pass

    def json_schema(self) -> ToolDescription:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": self.parameters,
            },
        }
```

In [4]:
# Define a custom tool that sums two numbers, it inherits from Tool
class SumTwoNumbers(Tool):
    def __init__(
        self,
        name: str,
        description: str,
        parameters: dict[str, Any],
    ):
        self.name = name
        self.description = description
        self.parameters = parameters

    # Define the logic of the tool, specify the type hints
    async def call(self, args: dict[str, Any]) -> dict[str, Any]:
        number_1 = args.get("number_1")
        number_2 = args.get("number_2")

        if not isinstance(number_1, (int, float)) or not isinstance(
            number_2, (int, float)
        ):
            raise ValueError("Both number_1 and number_2 must be int or float.")

        result = number_1 + number_2

        return {"result": result}

In [5]:
# Create an instance of the tool with appropriate metadata to describe its functionality
tool_sum = SumTwoNumbers(
    # Name, how it can be called
    name="sum_two_numbers",
    # What it does
    description="A tool to sum two numbers. It takes two numbers as input and returns their sum.",
    # What parameters it expects
    parameters={
        "type": "object",
        "properties": {
            # The first parameter
            "number_1": {
                # Type of the parameter
                "type": "number",
                # Description of the parameter
                "description": "The first number to be summed.",
            },
            # The second parameter
            "number_2": {
                "type": "number",
                "description": "The second number to be summed.",
            },
        },
        # Which parameters are required
        "required": ["number_1", "number_2"],
        # No additional properties allowed
        "additionalProperties": False,
    },
)

print(await tool_sum.call({"number_1": 5, "number_2": 10}))

{'result': 15}


# LLM using a tool

## Connect LLM to Tool

Once the tool defined in a format understandable for the LLM, and that can be used to constraint it's usage, it can be provided to the LLM.

In [6]:
query = "Sum the numbers 15 and 27 for me."
user_message = LLMMessage(role=Roles.USER, content=query)

# Define the LLM, but this time with the tool included
# "auto" tool choice lets the LLM decide when to use the tool (not just always use it)
llm = OpenAILLM(tools=[tool_sum], tool_choice="auto")

response = await llm.generate([user_message])

print("Role: ", response.role)
print("Content: ", response.content)
print("Tool Calls:")
for tool_call in response.tool_calls:
    print("- ", tool_call)

Role:  assistant
Content:  
Tool Calls:
-  id='call_XjioLCbUbflOkoOY9eQB0OPH' function=Function(name='sum_two_numbers', arguments='{"number_1":15,"number_2":27}') type='function'


## Run the Required Tools

Now that the LLM is able to ask to call the tool, we have to run the tools. 

Note: This step will typically be hidden by the next wrappers we will implement.

In [7]:
results = {}

# For each tool the LLM asks for
for tool_call in response.tool_calls:
    tool_name = tool_call.function.name
    tool_args = tool_call.function.arguments

    # Find the tool by name
    tool = next((t for t in llm.tools if t.name == tool_name), None)

    # If the tool is found, call it with the provided arguments
    if tool is not None:
        tools_args_json = json.loads(tool_args)
        tool_result = await tool.call(tools_args_json)

        # Save the result of the tool call
        results[tool_name] = tool_result

for tool_name, result in results.items():
    print(f"Result from tool '{tool_name}': {result}")

Result from tool 'sum_two_numbers': {'result': 42}


## Inform the LLM of the outcome

Finally the LLM can get back the outcome of the prediction. We are simply sending back text to it.

In [8]:
tools_answers = []

# For each tool result, create a LLMMessage to pass back to the LLM
for tool_call in response.tool_calls:
    tool_name = tool_call.function.name
    result = results[tool_name]
    call_id = tool_call.id

    tool_answer = LLMMessage(
        role=Roles.TOOL,
        name=tool_name,
        content=json.dumps(result),
        tool_call_id=call_id,
    )
    tools_answers.append(tool_answer)

In [ ]:
# Create the full conversation, including user message, LLM response, and tool answers
conversation = [user_message, response, *tools_answers]

In [10]:
# The LLM can now generate a final response based on the conversation
# Which includes the tool results
final_response = await llm.generate(conversation)

print("Role: ", final_response.role)
print("Content: ", final_response.content)

Role:  assistant
Content:  The sum of 15 and 27 is 42.


------------------------